<a href="https://colab.research.google.com/github/usama488/data-analysis/blob/main/notebooks/data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Set display options for pandas DataFrames
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Set a clean visualization style for matplotlib and seaborn
plt.style.use('seaborn-v0_8-whitegrid') # Using a seaborn style
sns.set_theme(style="whitegrid") # Setting seaborn theme

print("Libraries imported, display options set, and visualization style applied.")

Libraries imported, display options set, and visualization style applied.


In [13]:
# Data Loading
try:
    df = pd.read_csv('your_dataset.csv')
    print("Dataset loaded successfully.")

    print("\nDataset Shape:")
    display(df.shape)

    print("\nFirst 5 rows of the dataset:")
    display(df.head())

    print("\nData Types:")
    display(df.dtypes)

except FileNotFoundError:
    print("Error: 'your_dataset.csv' not found. Please make sure the file is in the correct directory or provide the full path.")
    df = None # Set df to None if file not found

Error: 'your_dataset.csv' not found. Please make sure the file is in the correct directory or provide the full path.


In [14]:
# Data Cleaning

if df is not None:
    print("Original dataset shape:", df.shape)

    # Check for missing values
    print("\nMissing values before cleaning:")
    display(df.isnull().sum())

    # Handle missing values (example: fill numerical with mean, categorical with mode)
    for col in df.columns:
        if df[col].isnull().any():
            if df[col].dtype in ['int64', 'float64']:
                df[col].fillna(df[col].mean(), inplace=True)
            else:
                df[col].fillna(df[col].mode()[0], inplace=True)

    print("\nMissing values after handling:")
    display(df.isnull().sum())

    # Drop duplicate rows
    initial_rows = df.shape[0]
    df.drop_duplicates(inplace=True)
    rows_after_dropping_duplicates = df.shape[0]
    print(f"\nNumber of duplicate rows dropped: {initial_rows - rows_after_dropping_duplicates}")
    print("Dataset shape after dropping duplicates:", df.shape)


    # Convert 'date' and 'datetime' columns to datetime objects
    for col in df.columns:
        if col.lower() == 'date' or col.lower() == 'datetime':
            try:
                df[col] = pd.to_datetime(df[col], errors='coerce')
                print(f"\nConverted '{col}' to datetime.")
                # Handle any values that couldn't be converted (they will be NaT - Not a Time)
                if df[col].isnull().any():
                    print(f"Warning: Some values in '{col}' could not be converted to datetime and were set to NaT.")
                    # You might want to handle these NaT values further, e.g., drop rows or fill with a placeholder
            except Exception as e:
                print(f"Could not convert column '{col}' to datetime: {e}")

    print("\nData Types after cleaning:")
    display(df.dtypes)

    df_cleaned = df.copy() # Create a cleaned copy for further steps
    print("\nData Cleaning steps completed.")
    print("\nFirst 5 rows of the cleaned dataset:")
    display(df_cleaned.head())

else:
    print("Data has not been loaded successfully. Please load the data first.")

Data has not been loaded successfully. Please load the data first.


In [15]:
# Exploratory Data Analysis (EDA)

if 'df_cleaned' in locals() and df_cleaned is not None:
    print("\nSummary Statistics:")
    display(df_cleaned.describe())

    # Histogram of a numeric column (replace 'your_numeric_column' with an actual column name)
    numeric_cols = df_cleaned.select_dtypes(include=np.number).columns
    if not numeric_cols.empty:
        col_to_plot = numeric_cols[0] # Use the first numeric column found
        print(f"\nHistogram of '{col_to_plot}':")
        plt.figure(figsize=(8, 5))
        sns.histplot(df_cleaned[col_to_plot], kde=True)
        plt.title(f'Distribution of {col_to_plot}')
        plt.xlabel(col_to_plot)
        plt.ylabel('Frequency')
        plt.show()
    else:
        print("\nNo numeric columns found for histogram.")


    print("\nCorrelation Heatmap:")
    plt.figure(figsize=(10, 8))
    # Select only numerical columns for correlation heatmap
    df_numeric = df_cleaned.select_dtypes(include=np.number)
    if not df_numeric.empty:
        sns.heatmap(df_numeric.corr(), annot=True, cmap='coolwarm', fmt=".2f")
        plt.title('Correlation Heatmap')
        plt.show()
    else:
        print("\nNo numeric columns found for correlation heatmap.")


    # Boxplot to detect outliers (replace 'your_numeric_column' with an actual column name)
    if not numeric_cols.empty:
        col_to_plot_boxplot = numeric_cols[0] # Use the first numeric column found
        print(f"\nBoxplot of '{col_to_plot_boxplot}' to detect outliers:")
        plt.figure(figsize=(8, 5))
        sns.boxplot(x=df_cleaned[col_to_plot_boxplot])
        plt.title(f'Boxplot of {col_to_plot_boxplot}')
        plt.xlabel(col_to_plot_boxplot)
        plt.show()
    else:
         print("\nNo numeric columns found for boxplot.")

    print("\nExploratory Data Analysis steps completed.")

else:
    print("Data has not been loaded and cleaned successfully. Please load and clean the data first.")

Data has not been loaded and cleaned successfully. Please load and clean the data first.


In [16]:
# Feature Engineering

if 'df_cleaned' in locals() and df_cleaned is not None:
    # Create 'profitmargin' feature
    # Ensure 'revenue' is not zero to avoid division by zero
    if 'profit' in df_cleaned.columns and 'revenue' in df_cleaned.columns:
        df_cleaned['profitmargin'] = np.where(df_cleaned['revenue'] != 0, df_cleaned['profit'] / df_cleaned['revenue'], 0)
        print("\n'profitmargin' feature created.")
    else:
        print("\nCould not create 'profitmargin' feature. Make sure 'profit' and 'revenue' columns exist.")

    # Extract month from any datetime column
    datetime_cols = df_cleaned.select_dtypes(include='datetime64[ns]').columns
    if not datetime_cols.empty:
        for col in datetime_cols:
            df_cleaned[f'{col}_month'] = df_cleaned[col].dt.month
            print(f"\nExtracted month from '{col}' into a new column '{col}_month'.")
    else:
        print("\nNo datetime columns found to extract month from.")

    print("\nFirst 5 rows of the updated dataset with new features:")
    display(df_cleaned.head())

    df_featured = df_cleaned.copy() # Create a copy after feature engineering
    print("\nFeature Engineering steps completed.")

else:
    print("Data has not been loaded and cleaned successfully. Please load and clean the data first.")

Data has not been loaded and cleaned successfully. Please load and clean the data first.


In [17]:
# Model Selection and Training (Linear Regression)

if 'df_featured' in locals() and df_featured is not None:
    # Assuming 'Advertising' is your independent variable (X) and 'Revenue' is your dependent variable (y)
    # Replace 'Advertising' and 'Revenue' with your actual column names
    try:
        X = df_featured[['Advertising']] # Independent variable(s)
        y = df_featured['Revenue'] # Dependent variable

        # Split data into training and testing sets (80/20 split)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        print("Data split into training and testing sets.")
        print(f"Training set shape (X_train, y_train): {X_train.shape}, {y_train.shape}")
        print(f"Testing set shape (X_test, y_test): {X_test.shape}, {y_test.shape}")

        # Initialize and train the Linear Regression model
        model = LinearRegression()
        model.fit(X_train, y_train)

        print("\nLinear Regression model trained successfully.")
        print(f"Model Intercept: {model.intercept_:.2f}")
        print(f"Model Coefficient (for Advertising): {model.coef_[0]:.2f}")

    except KeyError as e:
        print(f"Error: Make sure you have columns named 'Advertising' and 'Revenue' in your dataframe. Details: {e}")
    except NameError:
         print("Error: 'df_featured' is not defined. Please run the data loading, cleaning, and feature engineering steps first.")
else:
    print("Data has not been loaded and featured successfully. Please check the previous steps.")

Data has not been loaded and featured successfully. Please check the previous steps.


In [18]:
# Model Evaluation

if 'model' in locals() and 'X_test' in locals() and 'y_test' in locals():
    try:
        # Predict on the test set
        y_pred = model.predict(X_test)

        # Calculate Mean Squared Error
        mse = mean_squared_error(y_test, y_pred)

        # Calculate R2 Score
        r2 = r2_score(y_test, y_pred)

        print("\nModel Evaluation Metrics:")
        print(f"Mean Squared Error (MSE): {mse:.2f}")
        print(f"R2 Score: {r2:.2f}")

    except NameError:
        print("Error: Model or test data not found. Please ensure the Model Selection and Training step was completed successfully.")
    except Exception as e:
        print(f"An error occurred during model evaluation: {e}")
else:
    print("Model and test data are not available for evaluation. Please ensure the Model Selection and Training step was completed successfully.")

Model and test data are not available for evaluation. Please ensure the Model Selection and Training step was completed successfully.


In [19]:
# Scatter plot of actual vs predicted revenue

if 'y_test' in locals() and 'y_pred' in locals():
    try:
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.xlabel("Actual Revenue")
        plt.ylabel("Predicted Revenue")
        plt.title("Actual vs Predicted Revenue")
        plt.show()
    except Exception as e:
        print(f"An error occurred while creating the scatter plot: {e}")
else:
    print("Actual and predicted values are not available for plotting. Please ensure the Model Evaluation step was completed successfully.")

Actual and predicted values are not available for plotting. Please ensure the Model Evaluation step was completed successfully.


## Analysis Summary and Key Insights

This analysis aimed to understand the relationship between advertising and revenue and to build a linear regression model to predict revenue based on advertising spend.

**Key Findings:**

*   **Data Overview:** The dataset contained [Number] rows and [Number] columns. Initial data loading showed [Mention any initial observations, e.g., file not found issue].
*   **Data Cleaning:** Missing values were handled by [Explain how missing values were handled, e.g., filling with mean/mode]. Duplicate rows were [Mention if duplicates were dropped and how many]. Data types were [Mention any conversions, e.g., date columns to datetime].
*   **Exploratory Data Analysis (EDA):**
    *   Summary statistics revealed [Mention any interesting statistics like mean, median, standard deviation for key columns].
    *   The distribution of [Numeric Column Name] showed [Describe the distribution, e.g., skewed, normal].
    *   The correlation heatmap indicated [Describe key correlations, especially between advertising and revenue].
    *   Boxplots for [Numeric Column Name] showed [Mention presence or absence of outliers].
*   **Feature Engineering:** A new feature, `profitmargin`, was created as [Explain the calculation]. Month was extracted from [Date Column Name].
*   **Model Performance:**
    *   The Linear Regression model trained to predict Revenue from Advertising resulted in an R2 score of [Your R2 Score]. This indicates that [Interpret the R2 score, e.g., X% of the variance in Revenue can be explained by Advertising].
    *   The Mean Squared Error (MSE) of the model was [Your MSE Value]. This metric represents [Explain what MSE means in this context].
    *   The scatter plot of actual vs. predicted revenue values [Describe the plot, e.g., shows a clear linear trend, points are clustered around the line].
*   **Business Recommendations:** Based on this analysis, some recommendations could include:
    *   [Example: Increase advertising spend as there appears to be a positive correlation with revenue].
    *   [Example: Investigate outliers in [Column Name] as they might represent important events or data errors].
    *   [Example: Explore other features that might impact revenue based on correlations observed in EDA].
    *   [Any other relevant recommendations based on your specific data and business context].

**Next Steps:**

*   [Example: Explore more advanced regression models].
*   [Example: Gather more data on other potential factors influencing revenue].
*   [Example: Monitor the performance of the implemented recommendations].

In [20]:
import os

# Define the directory and filename
output_dir = 'data/processed'
output_filename = 'cleaned_sales.csv'
output_path = os.path.join(output_dir, output_filename)

if 'df_cleaned' in locals() and df_cleaned is not None:
    try:
        # Create the output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        # Save the cleaned dataframe to CSV
        df_cleaned.to_csv(output_path, index=False)

        print(f"Cleaned dataframe successfully saved to '{output_path}'")

    except Exception as e:
        print(f"An error occurred while saving the dataframe: {e}")
else:
    print("Cleaned dataframe ('df_cleaned') not found. Please ensure the data cleaning step was completed successfully.")

Cleaned dataframe ('df_cleaned') not found. Please ensure the data cleaning step was completed successfully.
